<a href="https://colab.research.google.com/github/mustafabozkaya/DeepLearningBootcamp2022/blob/master/Spectogram_Recognition_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[<img align="Left" width="100" height="100" src="https://thumbs.dreamstime.com/b/mb-initial-letter-vector-logo-icon-mb-initial-letter-vector-logo-icon-204517753.jpg">](https://github.com/mustafabozkaya)

# Spectogram Recognition with CNN

---
[<img align="Left" width="800" height="300" src="https://www.researchgate.net/publication/319081627/figure/fig1/AS:534034566004736@1504335170521/Spectrogram-of-a-speech-signal-with-breath-sound-marked-as-Breath-whose-bounds-are.png">](#)



**Colab** için kimlik doğrulama adımları:

**Drive yükleme işlemi**

**Drive da dosya konumlandırmayı yapma işlemleri**

In [1]:
from google.colab import drive
drive.mount('/content/mydrive/')

Mounted at /content/mydrive/


In [2]:
#unmount drive
# drive.flush_and_unmount()

In [22]:
!pwd


/content


In [23]:
%cd /content

/content


In [24]:
!pwd

/content


In [25]:
#Copying current content to new editable directory
#!cp -r "../content/drive/MyDrive/spectrograms/" "/sample_data/"


+
# Kurulum ve Kontroller

In [26]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5886906541365034509
 xla_global_id: -1]

In [ ]:
!cat /proc/meminfo

In [28]:
!pip install -q keras
!pip install -q Pillow

In [29]:
import tensorflow as tf
tf.test.gpu_device_name()


''

In [30]:
tf.__version__

'2.8.2'

In [31]:
#https://github.com/astrada/google-drive-ocamlfuse/
# !mkdir -p drive
# !google-drive-ocamlfuse drive
# !ls

In [32]:
!ls drive

ls: cannot access 'drive': No such file or directory


In [33]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras. layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import glob
import numpy as np
import os
import shutil
import cv2 as cv
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"]=(18,8)
%matplotlib inline
np.random.seed(42)

# Configuring İmage datasets folder 

In [41]:
dataset_dir=f"/content/mydrive/My Drive/spectrograms/"


In [42]:
for root, dirs, files in os.walk(dataset_dir, topdown=False):
    print(root)
    print(f"files type :{type(files)}")
    print(f"files lenth :{len(files)}")
    #print(os.path.join(root, name))
    print(f"DİR type :{type(dirs)}")
    print(f"DİR lenth :{len(dirs)}")
    
    

/content/mydrive/My Drive/spectrograms/6
files type :<class 'list'>
files lenth :374
DİR type :<class 'list'>
DİR lenth :0
/content/mydrive/My Drive/spectrograms/5
files type :<class 'list'>
files lenth :1000
DİR type :<class 'list'>
DİR lenth :0
/content/mydrive/My Drive/spectrograms/2
files type :<class 'list'>
files lenth :1000
DİR type :<class 'list'>
DİR lenth :0
/content/mydrive/My Drive/spectrograms/1
files type :<class 'list'>
files lenth :429
DİR type :<class 'list'>
DİR lenth :0
/content/mydrive/My Drive/spectrograms/3
files type :<class 'list'>
files lenth :1000
DİR type :<class 'list'>
DİR lenth :0
/content/mydrive/My Drive/spectrograms/8
files type :<class 'list'>
files lenth :929
DİR type :<class 'list'>
DİR lenth :0
/content/mydrive/My Drive/spectrograms/9
files type :<class 'list'>
files lenth :1000
DİR type :<class 'list'>
DİR lenth :0
/content/mydrive/My Drive/spectrograms/0
files type :<class 'list'>
files lenth :1000
DİR type :<class 'list'>
DİR lenth :0
/content/my

In [43]:
for root,dirname, filenames in os.walk(dataset_dir,):
    for filename in filenames:
        print(os.path.join(root, filename))

Streaming output truncated to the last 5000 lines.
/content/mydrive/My Drive/spectrograms/3/81722-3-0-21.png
/content/mydrive/My Drive/spectrograms/3/81722-3-0-26.png
/content/mydrive/My Drive/spectrograms/3/81722-3-0-28.png
/content/mydrive/My Drive/spectrograms/3/81791-3-0-0.png
/content/mydrive/My Drive/spectrograms/3/81791-3-0-1.png
/content/mydrive/My Drive/spectrograms/3/81791-3-1-0.png
/content/mydrive/My Drive/spectrograms/3/81799-3-0-1.png
/content/mydrive/My Drive/spectrograms/3/81799-3-0-0.png
/content/mydrive/My Drive/spectrograms/3/81799-3-1-0.png
/content/mydrive/My Drive/spectrograms/3/81799-3-1-2.png
/content/mydrive/My Drive/spectrograms/3/82024-3-0-4.png
/content/mydrive/My Drive/spectrograms/3/82024-3-0-5.png
/content/mydrive/My Drive/spectrograms/3/82024-3-0-6.png
/content/mydrive/My Drive/spectrograms/3/82024-3-0-8.png
/content/mydrive/My Drive/spectrograms/3/82811-3-0-1.png
/content/mydrive/My Drive/spectrograms/3/82811-3-3-0.png
/content/mydrive/My Drive/spectrog

In [44]:
!ls

mydrive  sample_data


# Data Preliminary Exploration

In [45]:
# create image datasets 

In [46]:
#  IMG_DIM = (150, 150)

train_files = glob.glob(str(os.path.join(dataset_dir,"1"))+'/*')


print(train_files)
print(f"1 spectogram img lenth :{len(train_files)}")
train_imgs = [img_to_array(load_img(img)) for img in train_files]
train_imgs = np.array(train_imgs)
train_labels = np.array([fn.split('/')[5].split(".")[0].strip() for fn in train_files])


print('Train dataset shape:', train_imgs.shape)
print('Train labels shape:', train_labels.shape)

['/content/mydrive/My Drive/spectrograms/1/100648-1-2-0.png', '/content/mydrive/My Drive/spectrograms/1/100648-1-1-0.png', '/content/mydrive/My Drive/spectrograms/1/100648-1-0-0.png', '/content/mydrive/My Drive/spectrograms/1/100648-1-3-0.png', '/content/mydrive/My Drive/spectrograms/1/100648-1-4-0.png', '/content/mydrive/My Drive/spectrograms/1/107090-1-1-0.png', '/content/mydrive/My Drive/spectrograms/1/107090-1-0-0.png', '/content/mydrive/My Drive/spectrograms/1/117536-1-0-0.png', '/content/mydrive/My Drive/spectrograms/1/118070-1-0-0.png', '/content/mydrive/My Drive/spectrograms/1/118496-1-0-0.png', '/content/mydrive/My Drive/spectrograms/1/118496-1-1-0.png', '/content/mydrive/My Drive/spectrograms/1/121299-1-0-0.png', '/content/mydrive/My Drive/spectrograms/1/125520-1-4-0.png', '/content/mydrive/My Drive/spectrograms/1/125520-1-1-0.png', '/content/mydrive/My Drive/spectrograms/1/125520-1-3-0.png', '/content/mydrive/My Drive/spectrograms/1/125520-1-0-0.png', '/content/mydrive/My Dr

In [47]:
train_labels[428]

'1'

In [48]:
image_size=(374,500)
batch_size=32

In [49]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=1007,
    image_size=image_size,
    batch_size=batch_size,
)

Found 8732 files belonging to 10 classes.
Using 6986 files for training.


In [50]:
train_ds.class_names

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [51]:
#Checking images and labels shapes (amount of images, height, width, color channels)
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 374, 500, 3)
(32,)


In [52]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=1007,
    image_size=image_size,
    batch_size=batch_size,
)

Found 8732 files belonging to 10 classes.
Using 6986 files for training.


In [53]:
#Checking images and labels shapes (amount of images, height, width, color channels)
for image_batch, labels_batch in test_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 374, 500, 3)
(32,)


### Veriden örnekleri görselleştirme

In [54]:
#Displaying image samples 
plt.figure(figsize=(18, 18))
for images, labels in train_ds.take(1):
    for i in range(batch_size):
        ax = plt.subplot(4,8,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

# Data Preparation

In [55]:
import os, sys, stat
#Defining parameters for the loader:
batch_size = 32
img_height = image_size[0]
img_width = image_size[1]

#Filtering out corrupted images
num_skipped = 0
for folder_name in (train_labels):
    folder_path = os.path.join(dataset_dir, folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()
        if not is_jfif:
            num_skipped += 1
            corrupted_dist="../content/sample_data/corrupted_img/"
            if not os.path.exists(corrupted_dist):
             os.mkdir(corrupted_dist)

            shutil.copy(fpath,corrupted_dist)
            # Delete corrupted image
            # os.chmod(fpath,stat.S_IXUSR)
            # os.remove(fpath)
            
print("Deleted %d images" % num_skipped)


Deleted 184041 images


NameError: ignored

In [ ]:
#Data augmentation
data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1)])

#Setting train/test split
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=(img_height, img_width),
    batch_size=batch_size)
ds_test = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=(img_height, img_width),
    batch_size=batch_size)


# Data Exploration

In [ ]:
#Checking images and labels shapes (amount of images, height, width, color channels)
for image_batch, labels_batch in ds_test:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
for image_batch, labels_batch in ds_train:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
#Displaying a random augmented image sample
plt.figure(figsize=(10, 10))
for images, _ in ds_test.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

# Modelling

In [ ]:
#Checking if the data format i.e the RGB channel is coming first or last so, whatever it may be, model will check first and then input shape will be feeded accordingly.
from keras import backend as K
if K.image_data_format() == "channels_first":
    input_shape = (3, img_height, img_width)
else:
    input_shape = (img_height, img_width, 3)

#Creating a model
model_dl = Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(input_shape)),
  layers.Conv2D(16, 3, padding="same", activation="relu"),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding="same", activation="relu"),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding="same", activation="relu"),
  layers.MaxPooling2D(),
  layers.Dropout(0.5),
  layers.Flatten(),
  layers.Dense(128, activation="relu"),
  layers.Dense(1, activation="sigmoid")
])

**Katlanların oluşturulması**

In [ ]:
model_dl.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))

 
model_dl.add(Conv2D(64, (3, 3), activation='relu'))


model_dl.add(MaxPooling2D(pool_size=(2, 2)))


model_dl.add(Dropout(0.25))


model_dl.add(Flatten())


model_dl.add(Dense(128, activation='relu'))

model_dl.add(Dropout(0.5))


model_dl.add(Dense(num_classes, activation='softmax'))

Modell Görselleştirme


In [ ]:
model_dl.summary()


In [ ]:

model_dl.compile(loss = keras.losses.categorical_crossentropy,
             optimizer = keras.optimizers.Adadelta(),
             metrics = ['accuracy'])

### 

In [ ]:
model_dl.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

In [ ]:
model_dl.save('save_models/mnist_model.h5')

In [ ]:
score = model_dl.evaluate(x_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test Accuracy:', score[1])


**Rastgele değer için test işlemi**

In [ ]:
model_test = model_dl.save('save_models/mnist_model.h5')

In [ ]:
test_image = x_test[32]
y_test[32]

In [ ]:
plt.imshow(test_image.reshape(28,28))

In [ ]:
test_data = x_test[32].reshape(1,28,28,1)
pre=model_test.predict(test_data, batch_size=1)

# Model Deployment